# Difusores

In this tutorial, you'll learn how to use models and schedulers to assemble a diffusion system for inference, starting with a basic pipeline and then progressing to the Stable Diffusion pipeline.

## Deconstruyendo el pipeline

Comencemos con el pipeline de Denoising Diffusion Probabilistic Models ([DDPM](https://arxiv.org/abs/2006.11239)) que permite modificar una imagen que solo contiene ruido para que gradualmente vaya creando una imagen.

In [ ]:
!pip install diffusers accelerate transformers --upgrade --quiet

Este es el pipeline que hace todo por nosotros:

In [ ]:
from diffusers import DDPMPipeline

ddpm = DDPMPipeline.from_pretrained("google/ddpm-cat-256").to("cuda")
image = ddpm(num_inference_steps=25).images[0]
image

Lo que hace es conbinar un modelo [UNet2DModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d#diffusers.UNet2DModel) con un scheduler [DDPMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/ddpm#diffusers.DDPMScheduler). El pipeline comienza con una imagen que es solo ruido y la pasa a traves del modelo tantas veces como dice el scheduler y va reduciendo el ruido en cada iteracion.

Para hacer este proceso paso a paso hacemos lo siguiente:

1. Cargamos el modelo y el scheduler:

In [ ]:
from diffusers import DDPMScheduler, UNet2DModel

scheduler = DDPMScheduler.from_pretrained("google/ddpm-cat-256")
model = UNet2DModel.from_pretrained("google/ddpm-cat-256").to("cuda")

2. Seleccionamos el numero de iteraciones:

In [ ]:
scheduler.set_timesteps(10)

3. La cantidad de scheduler timesteps crea un tensor con esa cantidad de elementos separados uniformemente. Cada elemento corresponde a un timestep en el que el modelo elimina ruedo de la imagen.

In [ ]:
scheduler.timesteps

4. Crear la imagen que contiene solamente ruido:

In [ ]:
from PIL import Image
import numpy as np
import torch

sample_size = model.config.sample_size
noise = torch.randn((1, 3, sample_size, sample_size)).to("cuda")

image = (noise / 2 + 0.5).clamp(0, 1)
image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
image = Image.fromarray((image * 255).round().astype("uint8"))
image

5. Crear un loop que en cada iteracion utilice el modelo UNet para estimar el residual de ruido y que el Scheduler tome esto como entrada para predecir la entrada de dicha iteracion.

In [ ]:
import matplotlib.pyplot as plt

input = noise

for t in scheduler.timesteps:
    with torch.no_grad():
        noisy_residual = model(input, t).sample
    previous_noisy_sample = scheduler.step(noisy_residual, t, input).prev_sample
    input = previous_noisy_sample

    image = (input / 2 + 0.5).clamp(0, 1)
    image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
    image = Image.fromarray((image * 255).round().astype("uint8"))
    image

    plt.figure(figsize=(2,3))
    plt.imshow(image)

Este es el proceso completo de eliminacion de ruido de un sistema de difusion.


## Deconstruyendo el pipeline de Stable Diffusion

Stable Diffusion es un modelo que convierte texto a imagen. Es llamado un modelo de *latent diffusion* porque funciona utilizando una representacion comprimida de menos dimensiones que la imagen.

Este pipeline es mas complejo que el anterior ya que contiene un Variational Auto Encoder para esta codificacion de la imagen, un codificador de texto y la UNet y el Scheduler vistos anteriormente.


Carguemos ahora todos estos componentes:

In [ ]:
from PIL import Image
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")
tokenizer = CLIPTokenizer.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="text_encoder")
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

Esta vez seleccionamos otro scheduler: [UniPCMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/unipc#diffusers.UniPCMultistepScheduler):

In [ ]:
from diffusers import UniPCMultistepScheduler

scheduler = UniPCMultistepScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

Usaremos GPU para que todo sea mas rapido:

In [ ]:
torch_device = "cuda"
vae.to(torch_device)
text_encoder.to(torch_device)
unet.to(torch_device)

### Crear text embeddings

Creamos los embeddings del texto que ingresaran como entrada a la UNet:

In [ ]:
prompt = ["a photograph of a panda bear playing golf"]
height = 512  # default height of Stable Diffusion
width = 512  # default width of Stable Diffusion
num_inference_steps = 25  # Number of denoising steps
guidance_scale = 7.5  # Scale for classifier-free guidance
generator = torch.manual_seed(0)  # Seed generator to create the inital latent noise
batch_size = len(prompt)

In [ ]:
text_input = tokenizer(
    prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
)

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

Tambien hay que generar los *unconditional text embeddings* para los tokens de padding. Estos tienen que tener la misma forma (`batch_size` y `seq_length`):

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer([""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt")
uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

Concatenamos ambos:

In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

### Crear ruido aleatorio

Ahora generamos el ruido la imagen latente es mas pequeña que la final pero eso esta bien porque el modelo se encargara de transformarla a 512x512


In [ ]:
latents = torch.randn(
    (batch_size, unet.in_channels, height // 8, width // 8),
    generator=generator,
)
latents = latents.to(torch_device)


El alto y el ancho se divide por 8 porque el modelo `vae` tiene 3 capas de down-sampling. Podemos chequear esto de la siguiente forma:

In [ ]:
2 ** (len(vae.config.block_out_channels) - 1)

### Eliminar el ruido paulatinamente

Start by scaling the input with the initial noise distribution, *sigma*, the noise scale value, which is required for improved schedulers like [UniPCMultistepScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/unipc#diffusers.UniPCMultistepScheduler):

In [ ]:
latents = latents * scheduler.init_noise_sigma

The last step is to create the denoising loop that'll progressively transform the pure noise in `latents` to an image described by your prompt. Remember, the denoising loop needs to do three things:

1. Set the scheduler's timesteps to use during denoising.
2. Iterate over the timesteps.
3. At each timestep, call the UNet model to predict the noise residual and pass it to the scheduler to compute the previous noisy sample.

In [ ]:
from tqdm.auto import tqdm

scheduler.set_timesteps(num_inference_steps)

for t in tqdm(scheduler.timesteps):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)

    latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, t, latents).prev_sample

### Decodificar la imagen

El ultimo paso es usar el vae para decodificar la representacion latente a una imagen:


In [ ]:
# escalar y decodificar la imagen latente con el vae
latents = 1 / 0.18215 * latents
with torch.no_grad():
    image = vae.decode(latents).sample

image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

## Proximos pasos

Un proximo paso interesante es poder hacer [finetuning](https://huggingface.co/docs/diffusers/v0.13.0/en/training/text2image) para incluir tus propias imagenes, como por ejemplo tus fotos. O tambien probar con un pipeline de [imagen a imagen](https://huggingface.co/tasks/image-to-image).

Sigue aprendiendo y experimentado en hugging face:

* Aprende a [crear tu propia pipeline](https://huggingface.co/docs/diffusers/main/en/using-diffusers/contribute_pipeline)

* Explora [otras pipelines](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/overview)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)

## Referencias

- [Diffusers](https://huggingface.co/docs/diffusers/quicktour)
- [Understanding pipelines, models and schedulers](https://huggingface.co/docs/diffusers/using-diffusers/write_own_pipeline)
- [How does Stable Diffusion work?](https://huggingface.co/blog/stable_diffusion#how-does-stable-diffusion-work)
- [Stable Diffusion from scratch](https://github.com/Animadversio/DiffusionFromScratch )
- [Andrew Chan Notes on Diffusion](https://andrewkchan.dev/posts/diffusion.html)